In [1]:
import json
import importlib
cim_profile = 'cim17v40'
from uuid import UUID
# cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)
import cimgraph.data_profile.cim17v40 as cim

In [2]:
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import GridappsdConnection
from cimgraph.models import FeederModel
import cimgraph.queries.sparql as sparql

# from cimgraph.models import DistributedArea

In [3]:
import cimgraph.utils as utils

In [4]:
params = ConnectionParameters(host = "localhost", port = "61613", cim_profile=cim_profile, iec61970_301=7)
gapps = GridappsdConnection(params)

In [5]:
utils.g

AttributeError: module 'cimgraph.utils' has no attribute 'g'

In [ ]:
feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=gapps, container=feeder, distributed=False)


In [ ]:
# feeder_mrids = ["_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62", # 13 bus
#                 "_C1C3E687-6FFD-C753-582B-632A27E28507",  # 123 bus
#                 "5B816B93-7A5F-B64C-8460-47C17D6E4B0F", # 13 bus asets
#                 "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3",  # 8500 node
#                 "_67AB291F-DCCD-31B7-B499-338206B9828F", # J1
#                 "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"]  # R2 12.47 3
# # feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node
# # feeder_mrid = "_9E985101-27AD-4FE4-B36A-EBECC98CDFAB" #MAPLE 10 bus
# for feeder_mrid in feeder_mrids:
#     feeder = cim.Feeder(mRID = feeder_mrid)
#     network = FeederModel(connection=gapps, container=feeder, distributed=False)
#     test_get_all_edges(network, cim)

In [ ]:
with open("topo_message.json") as topo:
    topo_msg = json.load(topo)

In [ ]:
# central_network = FeederModel(connection=gapps, container=feeder, distributed=False)

# dist_network_2 =  FeederModel(connection=gapps, container=feeder, distributed=True)

# dist_network_3 =  FeederModel(connection=gapps, container=feeder, distributed=True, topology_message=topo_msg)

In [ ]:
# for switch_area in dist_network_2.distributed_areas:
#     test_get_all_edges(switch_area, cim)
#     for sec_area in switch_area.distributed_areas:
#         test_get_all_edges(sec_area, cim)

In [ ]:
# for switch_area in dist_network_3.distributed_areas:
#     test_get_all_edges(switch_area, cim)
#     for sec_area in switch_area.distributed_areas:
#         test_get_all_edges(sec_area, cim)

In [ ]:
# from cimgraph.models import DistributedArea

# container = cim.EquipmentContainer(mRID=feeder_mrid)
# feeder_area = DistributedArea(container, gapps, distributed=True)
# feeder_area.build_from_topo_message(topo_msg['feeders'])

# feeder_area.get_all_edges(cim.ConnectivityNode)
# for node in feeder_area.graph[cim.ConnectivityNode].values():
#     print(node.name)

In [ ]:
# counter = 0
# for switch_topo in topo_msg['feeders']['switch_areas']:
#     print('switch area', counter)
#     container = cim.EquipmentContainer(mRID=feeder_mrid+'.'+str(counter))
#     SwitchArea = DistributedArea(container, gapps, distributed=True)
#     SwitchArea.build_from_topo_message(switch_topo)
#     counter = counter + 1

#     SwitchArea.get_all_edges(cim.ConnectivityNode)
#     for node in SwitchArea.graph[cim.ConnectivityNode].values():
#         print(node.name)

In [ ]:
# sw_counter = -1
# for switch_topo in topo_msg['feeders']['switch_areas']:
#     sw_counter +=1
#     sa_counter = -1
#     for sec_topo in switch_topo['secondary_areas']:
#         sa_counter +=1
#         area_id = str(sw_counter) + '.' + str(sa_counter)
#         print('sec area', area_id)
#         container = cim.EquipmentContainer(mRID=area_id)
#         SecondaryArea = DistributedArea(container, gapps, distributed=True)
#         SecondaryArea.build_from_topo_message(sec_topo)

#         SecondaryArea.get_all_edges(cim.ConnectivityNode)
#         for node in SecondaryArea.graph[cim.ConnectivityNode].values():
#             print(node.name)

In [ ]:
utils.get_all_line_data(network)
for phs in network.graph[cim.ACLineSegmentPhase].values():
    print(type(phs.phase))

In [ ]:
# network.get_all_edges(cim.ACLineSegment)
network.get_all_edges(cim.PerLengthPhaseImpedance)

In [ ]:
network.graph[cim.ACLineSegmentPhase][UUID('94fcafff-e5f7-4f97-b697-35c46656edca')].pprint()

In [ ]:
network.get_all_edges(cim.PerLengthImpedance)

In [ ]:
utils.get_all_data(network)

In [ ]:
query = (network.get_edges_query(cim.PerLengthPhaseImpedance))
print(query)

In [ ]:
x = cim.PhaseImpedanceData()
x.